In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2021-02-09 05:52:09--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  12.5MB/s    in 0.4s    

2021-02-09 05:52:09 (12.5 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

In [ ]:
!ls ml-100k/
df = pd.read_csv("ml-100k/u.data",sep='\t',names="user_emb_id,movie_emb_id,rating,timestamp".split(","))
len(df.user_emb_id.unique()), len(df.movie_emb_id.unique())
num_users = df['user_emb_id'].unique().max() + 1
num_movies = df['movie_emb_id'].unique().max() + 1
Rating = df.pivot( index = 'user_emb_id', columns ='movie_emb_id',values = 'rating').fillna(0)
print(Rating.shape)

allbut.pl  u1.base  u2.test  u4.base  u5.test  ub.base	u.genre  u.occupation
mku.sh	   u1.test  u3.base  u4.test  ua.base  ub.test	u.info	 u.user
README	   u2.base  u3.test  u5.base  ua.test  u.data	u.item
(943, 1682)


In [ ]:
Rating=np.array(Rating)

In [ ]:
import numpy
import sys

 
class MF(object):
    
 
    def __init__(self):
        self.Q = None
        self.P = None
        
 
    def fit(self,user_x_product, latent_features_guess=32, learning_rate=0.01, steps=20, regularization_penalty=0.01, convergeance_threshold=0.001):
     
        print ('training model...')
        return self.__factor_matrix(user_x_product, latent_features_guess, learning_rate, steps, regularization_penalty, convergeance_threshold)
 
    def predict_instance(self, row_index):
       
        return numpy.dot(self.P[row_index, :], self.Q.T)
 
    def predict_all(self):
       
        return numpy.dot(self.P, self.Q.T)
 
    def get_models(self):
       
        return self.P, self.Q
 
    def __factor_matrix(self, R, K, alpha, steps, beta, error_limit):
        """
        R = user x product matrix
        K = latent features count (how many features we think the model should derive)
        alpha = learning rate
        beta = regularization penalty (minimize over/under fitting)
        step = logistic regression steps
        error_limit = algo finishes when error reaches this level
        Returns:
        P = User x features matrix. (How strongly a user is associated with a feature)
        Q = Product x feature matrix. (How strongly a product is associated with a feature)
        To predict, use dot product of P, Q
        """
        # Transform regular array to numpy array
        print("K=",K)
        R = numpy.array(R)
 
        # Generate P - N x K
        # Use random values to start. Best performance
        N = len(R)
        M = len(R[0])
        P = numpy.random.rand(N, K)
 
        # Generate Q - M x K
        # Use random values to start. Best performance
        Q = numpy.random.rand(M, K)
        Q = Q.T
        error = 0
 
        # iterate through max # of steps
        for step in range(steps):
 
            # iterate each cell in r
            for i in range(len(R)):
                for j in range(len(R[i])):
                    if R[i][j] > 0:                   
                        # get the eij (error) side of the equation
                        eij = R[i][j] - numpy.dot(P[i, :], Q[:, j])

                        for k in range(K):
                            # (*update_rule) update pik_hat
                            P[i][k] = P[i][k] + alpha * (2* eij * Q[k][j] - beta * P[i][k]) 
                            # (*update_rule) update qkj_hat
                            #Q[k][j] = Q[k][j] + alpha * ( 2 * eij * P[i][k] - beta * Q[k][j] )
                            Q[k][j] = Q[k][j] + alpha * (  2* eij * P[i][k] - beta * (Q[k][j]))
          
        # track Q, P (learned params)
        # Q = Products x feature strength
        # P = Users x feature strength
        self.Q = Q.T
        self.P = P
 
      # self.__print_fit_stats(error, N, M)
 
    def __error(self, R, P, Q, K, beta):
        """
        Calculates the error for the function
        :param R:
        :param P:
        :param Q:
        :param K:
        :param beta:
        :return:
        """
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                   # i=i+1
                 #   j=j+1
                    # loss function error sum( (y-y_hat)^2 )
                    e = e + pow(R[i][j]-numpy.dot(P[i,:],Q[:,j]), 2)
 
                    # add regularization
                    for k in range(K):
                     #   k=k+1
 
                        # error + ||P||^2 + ||Q||^2
                        e = e + (beta/2) * ( pow(P[i][k], 2) + pow(Q[k][j], 2) )
        return e
 
    def rmse(self,R):
        """
        A function to compute the total mean square error
        """
        xs, ys = R.nonzero()
        predicted = self.predict_all()
        count=0
        error = 0
        for x, y in zip(xs, ys):
            count=count+1
            error += pow((R[x, y] - predicted[x, y]), 2)
        return np.sqrt(error/count)
    def mae(self,R):
 
      xs, ys = R.nonzero()
      predicted = self.predict_all()
      count=0
      error = 0
      for x, y in zip(xs, ys):
 
        count=count+1
        error += abs((R[x, y] - predicted[x, y]))
      return (error/count)
    def __print_fit_stats(self, error, samples_count, products_count):
        print ('training complete...')
        print ('------------------------------')
        print ('Stats:')
        print ('Error: %0.2f' % error)
        print ('Samples: ' + str(samples_count))
        print ('Products: ' + str(products_count))
        print ('------------------------------')

In [ ]:

 
 
class MF_Theta(object):
    
 
    def __init__(self):
        self.Q = None
        self.P = None
        
 
    def fit(self,user_x_product,theta, latent_features_guess=32, learning_rate=0.01, steps=20, regularization_penalty=0.1, convergeance_threshold=0.001):
     
        print ('training model...')
        return self.__factor_matrix(user_x_product,theta, latent_features_guess, learning_rate, steps, regularization_penalty, convergeance_threshold)
 
    def predict_instance(self, row_index):
       
        return numpy.dot(self.P[row_index, :], self.Q.T)
 
    def predict_all(self):
       
        return numpy.dot(self.P, self.Q.T)
 
    def get_models(self):
       
        return self.P, self.Q
 
    def __factor_matrix(self, R,theta, K, alpha, steps, beta, error_limit):
        """
        R = user x product matrix
        K = latent features count (how many features we think the model should derive)
        alpha = learning rate
        beta = regularization penalty (minimize over/under fitting)
        step = logistic regression steps
        error_limit = algo finishes when error reaches this level
        Returns:
        P = User x features matrix. (How strongly a user is associated with a feature)
        Q = Product x feature matrix. (How strongly a product is associated with a feature)
        To predict, use dot product of P, Q
        """
        # Transform regular array to numpy array
        R = numpy.array(R)
 
        # Generate P - N x K
        # Use random values to start. Best performance
        N = len(R)
        M = len(R[0])
        P = numpy.random.rand(N, K)
 
        # Generate Q - M x K
        # Use random values to start. Best performance
        Q = numpy.random.rand(M, K)
        Q = Q.T
        error = 0
 
        # iterate through max # of steps
        for step in range(steps):
 
            # iterate each cell in r
            for i in range(len(R)):
                for j in range(len(R[i])):
                    if R[i][j] > 0:                   
                        # get the eij (error) side of the equation
                        eij = R[i][j] - numpy.dot(P[i, :], Q[:, j])

                        for k in range(K):
                            # (*update_rule) update pik_hat
                            P[i][k] = P[i][k] + alpha * ( 2 *eij * Q[k][j]-beta * P[i][k]) 
                            # (*update_rule) update qkj_hat
                            Q[k][j] = Q[k][j] + alpha * (  2 *eij * P[i][k] - beta * (Q[k][j]-theta[k][j] ))
          
        self.Q = Q.T
        self.P = P
 
 
  
 
    def rmse(self,R):
        """
        A function to compute the total mean square error
        """
        xs, ys = R.nonzero()
        predicted = self.predict_all()
        count=0
        error = 0
        for x, y in zip(xs, ys):
            count=count+1
            error += pow((R[x, y] - predicted[x, y]), 2)
        return np.sqrt(error/count)
    def mae(self,R):
 
      xs, ys = R.nonzero()
      predicted = self.predict_all()
      count=0
      error = 0
      for x, y in zip(xs, ys):
 
        count=count+1
        error += abs((R[x, y] - predicted[x, y]))
      return (error/count)
    def __print_fit_stats(self, error, samples_count, products_count):
        print ('training complete...')
        print ('------------------------------')
        print ('Stats:')
        print ('Error: %0.2f' % error)
        print ('Samples: ' + str(samples_count))
        print ('Products: ' + str(products_count))
        print ('------------------------------')

In [ ]:
#read 1682_32 file after AE Bert 'plot_vec_AE_1682_32'
from google.colab import files
uploaded = files.upload()

Saving plot_vec_AE_1682_20.npy to plot_vec_AE_1682_20.npy


In [ ]:
theta=np.load('plot_vec_AE_1682_20.npy',allow_pickle=True)

In [ ]:
print(np.max(theta))

0.01


In [ ]:
theta=theta.T
theta=theta/(100*(np.max(theta)))

In [ ]:
print(theta.shape)

(20, 1682)


In [ ]:
#theta=theta/(100*(np.max(theta)))
b=MF_Theta()    
b.fit(Rating,theta,latent_features_guess=20,learning_rate=0.01,steps=20,regularization_penalty=0.01)
print(b.mae(Rating))
print(b.rmse(Rating))

training model...
0.5261958593374566
0.6754241701055053


In [ ]:
theta=theta/(2*(np.max(theta)))
b=MF_Theta()    
b.fit(Rating,theta,latent_features_guess=20,learning_rate=0.01,steps=20,regularization_penalty=0.01)
print(b.mae(Rating))
print(b.rmse(Rating))

training model...
0.5355292349107654
0.6867603031344912


In [ ]:
p,q=b.get_models()
print(p,q)
print(np.mean(p),np.mean(q),np.mean(theta))

[[-0.13292068  0.22052411  0.43796086 ... -0.07909872  0.56106619
   0.57080318]
 [ 0.1240996   0.08566591  0.45472665 ...  0.87149068  0.16946167
  -0.02017255]
 [-0.05808671  0.05361845 -0.25337746 ... -0.12569537  0.51387057
   0.86073635]
 ...
 [ 0.16087699  0.47356689  0.45539861 ...  0.6628524  -0.0935433
   0.3464694 ]
 [ 0.33726673  0.24901774  0.94642839 ...  0.53318596  0.17122523
   0.55958961]
 [-0.23011694  0.61106668 -0.00223909 ...  0.9443108   0.7728721
   0.67541346]] [[0.74669541 1.45241735 0.33861532 ... 0.20141921 0.22971427 0.18707113]
 [0.58925402 0.81406868 0.80963912 ... 0.71178378 0.4610337  1.12604677]
 [0.44346294 0.33531385 0.12945276 ... 0.11714786 0.59755977 1.23479511]
 ...
 [0.88783059 0.42256466 0.19391214 ... 0.6825064  0.17094988 0.16704578]
 [0.91195792 0.68925828 0.87417139 ... 0.51338412 0.72111233 0.19348224]
 [0.53607686 0.18136591 0.38760599 ... 0.94006199 0.51896132 0.06547124]]
0.3342173136034209 0.6246386086753966 0.74798006


In [ ]:
a=MF() 
a.fit(Rating,latent_features_guess=20,learning_rate=0.01,steps=20,regularization_penalty=0.01) 
print(a.mae(Rating))
print(a.rmse(Rating))

training model...
K= 20
0.5316989430548235
0.6810256329051676


In [ ]:
a=MF() 
a.fit(Rating,latent_features_guess=,learning_rate=0.01,steps=20,regularization_penalty=0.1) 
print(a.mae(Rating))
print(a.rmse(Rating))

training model...
K= 15
0.5966686482121042
0.757405058296339


In [ ]:
c=MF_Theta()    
c.fit(Rating,theta,latent_features_guess=5,learning_rate=0.01,steps=50,regularization_penalty=0.1)
print(c.mae(Rating))
print(c.rmse(Rating))

training model...
0.7118158596331274
0.898479270869477


In [ ]:
d=MF() 
d.fit(Rating,latent_features_guess=5,learning_rate=0.01,steps=50,regularization_penalty=0.1) 
print(d.mae(Rating))
print(d.rmse(Rating))

training model...
0.662325548197847
0.8415399078255104


In [ ]:
#Set the number of values to replace. For example 20%:
prop = int(R.size * 0.2)
print("R.size",R.size)
#Randomly choose indices of the numpy array:
print(R.shape)
I=[]
J=[]
for k in (range(prop)):
  i=np.random.choice(R.shape[0])
  j=np.random.choice(R.shape[1])
  
  if(R[i,j]>0):       
    R[i,j]=0
    I.append(i)
    J.append(j)
    
#print("Original:\n",R1)
#print("Test Set:\n",R)
R=np.rint(R)
print(I)
print(J)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(R, R1)

print("MSE=",mse**0.5)

print("\nTraining ...\n")

a = MF_Theta()
a.fit(R,R)

print("\nDone\n")

R.size 1586126
(943, 1682)
[434, 898, 862, 566, 43, 408, 267, 912, 863, 448, 915, 59, 473, 596, 759, 57, 274, 748, 814, 932, 784, 333, 254, 197, 262, 869, 179, 681, 373, 777, 694, 23, 103, 740, 664, 929, 784, 647, 255, 384, 68, 275, 537, 234, 647, 444, 790, 222, 803, 61, 641, 535, 889, 646, 536, 670, 431, 879, 6, 585, 82, 880, 550, 236, 499, 208, 869, 620, 558, 853, 446, 216, 393, 391, 398, 880, 873, 84, 842, 611, 293, 267, 684, 532, 20, 550, 386, 41, 275, 189, 28, 794, 199, 369, 107, 667, 531, 213, 402, 377, 275, 397, 879, 397, 256, 462, 249, 693, 249, 388, 89, 379, 406, 586, 477, 631, 598, 486, 556, 806, 282, 450, 575, 404, 471, 118, 586, 548, 768, 80, 814, 133, 541, 5, 500, 457, 893, 471, 617, 342, 270, 620, 313, 267, 616, 212, 267, 120, 48, 152, 866, 84, 644, 874, 617, 235, 425, 232, 361, 324, 384, 611, 659, 415, 448, 285, 74, 499, 0, 507, 197, 275, 827, 193, 159, 871, 292, 715, 734, 450, 726, 118, 444, 129, 6, 278, 885, 773, 312, 224, 726, 200, 323, 312, 338, 566, 205, 151, 255, 3

In [ ]:
print(a.mae(R))
print(a.rmse(R))